# CLTV - SHOPIFY Shortened.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
import warnings
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot 
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model, preprocessing
%matplotlib inline
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
## We create our dataframe with the raw data.
raw_data = pd.read_csv('.\CLEANED_data.csv')
raw_data.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment Terms Name,Next Payment Due At
0,#29489,Anonymous4245,paid,11/11/2021 16:53,unfulfilled,NaN,no,USD,142.0,0.00,11.72,153.72,NaN,0.0,Free Shipping,11/11/2021 16:53,1,Mint Crossback Sports Bra - S,38.0,NaN,100.0,True,True,pending,NaN,NaN,NaN,NaN,NaN,DIXON,'61021,IL,US,NaN,NaN,NaN,NaN,NaN,NaN,DIXON,'61021,IL,US,NaN,NaN,NaN,NaN,Shopify Payments,c29208367530214.1,0.0,NaN,4.550000e+12,NaN,Low,web,0.0,Dixon City Tax 0.5%,0.71,Illinois State Tax 6.25%,8.88,Lee County Tax 1.5%,2.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illinois,Illinois,NaN,NaN
1,#29489,Anonymous4245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/11/2021 16:53,1,Black/Burgundy Contour Leggings - XS,52.0,NaN,300.0,True,True,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#29489,Anonymous4245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/11/2021 16:53,1,Black/Mint Contour Leggings - XS,52.0,NaN,100.0,True,True,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#29488,Anonymous9987,paid,11/11/2021 10:09,unfulfilled,NaN,no,USD,40.0,5.36,2.90,48.26,NaN,0.0,Standard Shipping,11/11/2021 10:09,1,Black Long Sleeve Top - L,40.0,NaN,102.0,True,True,pending,NaN,NaN,NaN,NaN,NaN,Moreno Valley,'92553,CA,US,NaN,NaN,NaN,NaN,NaN,NaN,Moreno Valley,'92553,CA,US,NaN,NaN,NaN,NaN,Afterpay North America,c29207688708326.1,0.0,NaN,4.550000e+12,NaN,Low,web,0.0,California State Tax 7.25%,2.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California,California,NaN,NaN
4,#29487,Anonymous9675,paid,11/10/2021 14:54,fulfilled,11/11/2021 10:56,no,USD,94.0,5.06,0.00,99.06,NaN,0.0,Standard Shipping,11/10/2021 14:54,1,Raspberry Crossover Sports Bra - XL,38.0,NaN,53.0,True,True,fulfilled,NaN,NaN,NaN,NaN,NaN,ELKINS PARK,19027-1829,PA,US,NaN,NaN,NaN,NaN,NaN,NaN,ELKINS PARK,19027-1829,PA,US,NaN,NaN,NaN,NaN,Shopify Payments,c29206160441574.1,0.0,NaN,4.550000e+12,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pennsylvania,Pennsylvania,NaN,NaN


In [3]:
# We now look at the columns to get more information about them. 
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50418 entries, 0 to 50417
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Name                         50418 non-null  object 
 1   Email                        46372 non-null  object 
 2   Financial Status             28489 non-null  object 
 3   Paid at                      27735 non-null  object 
 4   Fulfillment Status           28489 non-null  object 
 5   Fulfilled at                 27256 non-null  object 
 6   Accepts Marketing            28489 non-null  object 
 7   Currency                     28489 non-null  object 
 8   Subtotal                     28489 non-null  float64
 9   Shipping                     28489 non-null  float64
 10  Taxes                        28489 non-null  float64
 11  Total                        28489 non-null  float64
 12  Discount Code                7339 non-null   object 
 13  Discount Amount 

In [ ]:
## We create a list of columns to drop and use the drop data to remove them from our dataframe.
columns_to_drop = ['Fulfilled at','Currency','Created at','Lineitem compare at price','Lineitem requires shipping','Lineitem taxable','Lineitem fulfillment status','Billing Name','Billing Street','Billing Address1','Billing Address2','Billing Company','Billing Phone','Shipping Name','Shipping Street','Shipping Address1','Shipping Address2','Shipping Company','Shipping City','Shipping Zip','Shipping Province','Shipping Country','Shipping Phone','Notes','Note Attributes','Payment Reference','Vendor','Id','Tags','Risk Level','Source','Tax 1 Name','Tax 1 Value','Tax 2 Name','Tax 2 Value','Tax 3 Name','Tax 3 Value','Tax 4 Name','Tax 4 Value','Tax 5 Name','Tax 5 Value','Phone','Receipt Number','Duties','Billing Province Name','Shipping Province Name','Payment Terms Name','Next Payment Due At','Lineitem sku']
raw_data.drop(columns=columns_to_drop, inplace = True)
raw_data = raw_data.rename(columns={'Name': 'OrderID','Paid at': 'Transaction Date','Email':'CustomerID'} )
# We use the fillna method to fill "NaN" values with either empty strings or 0s to clean our data.
raw_data['Discount Code'] = raw_data['Discount Code'].fillna('')
raw_data['Discount Amount'] = raw_data['Discount Amount'].fillna(0)
raw_data.dropna(subset=['CustomerID'], inplace = True)
raw_data['Transaction Date'] = pd.to_datetime(raw_data['Transaction Date'], errors='coerce')
raw_data.head(3)

In [ ]:
## Let's see how many unique values will be in each dataframe. 
print('The number of unique orders in this dataset is: ' + str(raw_data['OrderID'].nunique()))
print('The number of unique customers in this dataset is: ' + str(raw_data['CustomerID'].nunique()))
print('The number of unique items sold in this dataset is: ' +str(raw_data['Lineitem name'].nunique()))

## Creating Seperate Dataframes. 

In [ ]:
## Prior to creating our dataframe, we define functions which we will be used to retrieve infromation we want below. 
def count_discount_codes(discount_codes):
    '''
    This function returns the number of distinct coupons used 
    '''
    counter = 0
    for entry in discount_codes:
        if entry != '':
            counter += 1
    return counter

def time_elapsed(x):
    '''
    This function returns the number of days between a customer's first order and most recent order.
    '''
    return (x.max() - x.min()).days

def order_freq(x):
    '''
    This function returns the how often a customer makes a purchase (in days)
    '''
    return (x.max() - x.min()).days / x.count()

def repeat_customer(x):
    '''
    This function return whether or not the customer is a repeat customer. 
    '''
    if x.max() != x.min():
        return 1
    else:
        return 0

In [ ]:
## We create the customer dataframe. 
datelimit = raw_data['Transaction Date'].max() - timedelta(days=365)
raw_data = raw_data[raw_data['Transaction Date'] < datelimit]
customer_cltv =  raw_data.groupby(['CustomerID']).agg(Number_of_Orders = ('OrderID', 'nunique'),
                                            Total_Items_Purchased = ('Lineitem quantity', np.sum),
                                            Total_Spent = ('Subtotal', np.sum),
                                            Smallest_purchase = ('Subtotal', min),
                                            Largest_purchase = ('Subtotal', max),
                                            Average_order_spend = ('Subtotal', np.mean),
                                            Coupons_Used = ('Discount Code', count_discount_codes),                         
                                            First_Order = ('Transaction Date', np.min), 
                                            Most_Recent_Order = ('Transaction Date', np.max),
                                            Time_Elapsed_days = ('Transaction Date', time_elapsed),
                                            Frequency_in_days =('Transaction Date', order_freq),
                                            repeat_customer =('Transaction Date', repeat_customer)
                                            )

# We sort the values to get the customers who placed the most orders first. 
customer_cltv.sort_values('Number_of_Orders', ascending = False, inplace=True)
customer_cltv['cltv_start'] = customer_cltv['First_Order'] + timedelta(days=7)
customer_cltv['cltv_end'] = customer_cltv['First_Order'] + timedelta(days=365)
customer_cltv = customer_cltv[customer_cltv['CustomerID'] != '#REF!']

## We drop rows for which we don't have data and preview our customer table. 
customer_cltv.dropna(inplace=True)
customer_cltv.reset_index(inplace=True)
customer_cltv.sort_values('CustomerID').head(3)

### Creating a unique order dataframe.

In [ ]:
# We create the orders dataframe. 
orders =  raw_data.groupby(['OrderID']).agg(Total_Spent = ('Subtotal', np.sum),
                                            Total_Items_Purchased = ('Lineitem quantity', np.sum),
                                            CustomerID = ('CustomerID', np.max),
                                            Transaction_Date = ('Transaction Date', np.max)
                                            )

orders.reset_index(inplace= True)

## We remove the '#' from OrdersID and change it to int type. 
orders['OrderID'] = orders['OrderID'].apply(lambda x: x.replace('#','')).astype(int)

## We sort by OrderID, and preview the dataframe. 
orders = orders.sort_values('OrderID', ascending = False)
orders.dropna(inplace=True)

## We add a column to identify the customers who made a repeat purchase.
duplicated_orders = list(orders.CustomerID[orders.CustomerID.duplicated()])
orders['repeat_customer'] = np.where(orders['CustomerID'].apply(lambda x: x in duplicated_orders),1,0)
orders.head(3)


### Creating a unique item dataframe.

In [ ]:
# We create our items dataframe. 
items =  raw_data.groupby(['Lineitem name']).agg(Times_Purchased = ('Lineitem quantity', np.sum),
                                                 Price = ('Lineitem price', np.max)
                                                 )

items.sort_values('Times_Purchased', ascending = False, inplace=True)
items.reset_index(inplace=True)

## We create a size column seperating it in 6 different sizes and a no size. 
"""
Filling item info: 
 - left column is the desired label (for example for sizes, 'L', 'M' etc.)
 - right column between [] is the list (separated by commas) of ways the info can appear in the item's name
See example below on how to fill for item sizes if the sizes show up as "... / L" or "... - L", etc.
"""

def get_item_size(row):
    dict_sizes = {'XS': ['- XS', '/ XS'],
                  'S': ['- S', '/ S'],
                  'M': ['- M', '/ M'],
                  'L': ['- L', '/ L'],
                  'XL': ['- XL', '/ XL'],
                  'XXL': ['- XXL', '/ XXL']}

    item_name = row['Lineitem name']
    for key in dict_sizes.keys():
        for value in dict_sizes[key]:
            if value in item_name:
                return key
    return 'No size'

items['size'] = items.apply(get_item_size, axis=1)

## We create an item type column and seperate in 12 different products and other. 
"""
The method is the same for the size method above, but this time with product type. 
"""


def get_item_type(row):
    dict_items = {'Long Sleeve': ['Long Sleeve'],
                  'Barbell': ['Barbell'],
                  'Cable Cuffs': ['Cable Cuffs'],
                  'Resistance Bands': ['Resistance Bands'],
                  'Leggings': ['Leggings'],
                  'Sports Bra': ['Sports Bra'],
                  'Jacket': ['Jacket'],
                  'Crop Tops': ['Crop Tops'],
                  'Shorts': ['Shorts'],
                  'Shirts': ['Shirts'],
                  'Sweater': ['Sweater'],
                  'Hoodie': ['Hoodie'],
                  'Camo': ['Camo']}
    
    item_name = row['Lineitem name']
    for key in dict_items.keys():
        for value in dict_items[key]:
            if value in item_name:
                return key
    return 'Other'
    
items['type'] = items.apply(get_item_type, axis=1)

## We create a unique product sku for each of these items. 
items['Product Sku'] = [i+1 for i in range(len(items))]

## We rearrange the columns so that Product Sku becomes our first column and preview. 
items = items.iloc[:, [5, 0, 1, 2, 3, 4]]
items.head(5)

## Preparing the Data for CLTV Analysis. 

As a result from these visualizations, we are able to see that it is appropriate to use one year interval for our CLTV analysis. Let's create this. 

In [ ]:
## We define our date_limit for the analyis. We will remove data before this date. 

date_limit = customer['Most_Recent_Order'].max() - timedelta(days=365)
print('The most recent date in our dataset is '+ str(customer['Most_Recent_Order'].max()) + ' therefore, our date limit is: ' + str(date_limit) )

## Creating a CLTV customers dataframe: 
customer_cltv = customer[customer['First_Order'] < date_limit]

# For each customer, we create a cltv_start date and a cltv end date. 
customer_cltv['cltv_start'] = customer_cltv['First_Order'] + timedelta(days=7)
customer_cltv['cltv_end'] = customer_cltv['First_Order'] + timedelta(days=365)

customer_cltv = customer_cltv[customer_cltv['CustomerID'] != '#REF!']

## Let's preview the first 3 rows. 
print('Our previous customer table contained ' + str(customer.CustomerID.count()) + ' different customers. After removing those who made their first purchase after November 11th 2020, we are left with ' + str(customer_cltv.CustomerID.count()))
customer_cltv.sort_values('CustomerID').head(3)

Now that we have our new customers table which has the newly created cltv start and end dates for each customer, we will merge the customer stats table with the raw data clv table and the items table so that for each row, we get the cltv dates and item breakdowns for all the customers.

In [ ]:
## We merge our customer_stats_cltv with both the raw_data as well as the items tables. 
merged_data = raw_data.merge(customer_cltv, left_on = 'CustomerID', right_on = 'CustomerID').sort_values('CustomerID')
merged_data = merged_data.merge(items, left_on = 'Lineitem name', right_on = 'Lineitem name').sort_values('CustomerID')
print('The total number of items purchased before the limit date of Nov 11th 2020 is ' +str(merged_data['OrderID'].count()))
merged_data.head(3)

Now for each transaction, we have thei cltv start and end dates of their respective customers as well as the breakdown of sizes and types of item. Now we want to filter this data so that we get transactions that fall within the one year cltv period for each customer. Additionally, we will want to track how many of these orders were made within the first week. The data then gets filtered: 

In [ ]:
## Now we want to filter cltv_data so that all the items ordered are within our cltv dates. 
cltv_data  = merged_data[(merged_data ['Transaction Date'] >= merged_data ['First_Order']) & (merged_data ['Transaction Date'] <= merged_data ['cltv_end'])]
cltv_data_one_week  = cltv_data[(cltv_data ['Transaction Date'] >= cltv_data ['First_Order']) & (cltv_data ['Transaction Date'] <= cltv_data ['cltv_start'])]
print('The number of items purchased that fall within the one year CLTV will be ' + str(cltv_data['OrderID'].count()))
print('The number of items purchased that fall within the first week of the one year CLTV will be ' + str(cltv_data_one_week['OrderID'].count()))
cltv_data.head(2)

Our features variables are currently categorized and we want tidy data. Below, we will create a function that will be able to perform one hot encoding, where we replace the categorical values in size and type to and assign 0 if this is not the case for the transaction and 1 if it is. 

In [ ]:
## We define the function and apply it to both the cltv_data and the one week cltv. 
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

cltv_data = encode_and_bind(cltv_data, 'size')
cltv_data = encode_and_bind(cltv_data, 'type')
cltv_data_one_week = encode_and_bind(cltv_data_one_week , 'size')
cltv_data_one_week  = encode_and_bind(cltv_data_one_week , 'type')
cltv_data.head(2)

In [ ]:
cltv_data_one_week.head(2)

Now that we have performed the encoding and modified the columns, the next step will be to re-aggregate cltv_data and cltv_one_week data by customer now that we have all the information, then perform one last merge where we will be able to have information on the transactions during both each customers CLTV period and the transactions within one week after each customer's first order. 

In [ ]:
## We start by creating the customer CLTV table. 
customer_cltv = cltv_data.groupby('CustomerID').agg(
            CLTV_Total_Spent = ('Subtotal', sum),
            Smallest_Purchase = ('Subtotal', min),    
            Largest_Purchase = ('Subtotal', max),
            Average_order_spend = ('Subtotal', np.mean),
            Number_of_orders = ('Subtotal', 'nunique'),
            First_Order = ('Transaction Date', min),   
            Last_Order = ('Transaction Date', max),
            Time_Elapsed = ('Transaction Date', time_elapsed),
            Frequency_in_days =('Transaction Date', order_freq),
            cltv_start =('cltv_start', max),
            cltv_end =('cltv_end', max),
            repeat_customer =('repeat_customer', max),
            Size_XSmall =('size_XS', max),
            Size_Small =('size_S', max),
            Size_Medium =('size_M', max),
            Size_Large =('size_L', max),
            Size_XLarge =('size_XL', max),
            No_Size =('size_No size', max),
            Type_Barbell =('type_Barbell', max),
            Type_CableCuffs =('type_Cable Cuffs', max),
            Type_CropTops =('type_Crop Tops', max),
            Type_Hoodie =('type_Hoodie', max),
            Type_Jacket =('type_Jacket', max),
            Type_Leggings =('type_Leggings', max),
            Type_LongSleeve =('type_Long Sleeve', max),
            Type_ResistanceBands =('type_Resistance Bands', max),
            Type_Shirts =('type_Shirts', max),
            Type_Shorts =('type_Shorts', max),
            Type_SportsBra =('type_Sports Bra', max),
            Type_Sweater =('type_Sweater', max),
            Type_Other =('type_Other', max)).reset_index()
customer_cltv.head(2)

In [ ]:
## Now we do the same but for orders only in the first week. 
customer_cltv_first_week = cltv_data_one_week.groupby('CustomerID').agg(
            CLTV_one_Week = ('Subtotal', sum),
            Average_spend_one_week = ('Subtotal', np.mean),
            Number_of_orders_one_week = ('Subtotal', 'nunique'),
            First_Order = ('Transaction Date', min),   
            Last_Order = ('Transaction Date', max),
            Frequency_in_days =('Transaction Date', order_freq),
            cltv_start =('cltv_start', max),
            cltv_end =('cltv_end', max),
            fw_Size_XSmall =('size_XS', max),
            fw_Size_Small =('size_S', max),
            fw_Size_Medium =('size_M', max),
            fw_Size_Large =('size_L', max),
            fw_Size_XLarge =('size_XL', max),
            fw_No_Size =('size_No size', max),
            fw_Type_Barbell =('type_Barbell', max),
            fw_Type_CableCuffs =('type_Cable Cuffs', max),
            fw_Type_CropTops =('type_Crop Tops', max),
            fw_Type_Hoodie =('type_Hoodie', max),
            fw_Type_Jacket =('type_Jacket', max),
            fw_Type_Leggings =('type_Leggings', max),
            fw_Type_LongSleeve =('type_Long Sleeve', max),
            fw_Type_ResistanceBands =('type_Resistance Bands', max),
            fw_Type_Shirts =('type_Shirts', max),
            fw_Type_Shorts =('type_Shorts', max),
            fw_Type_SportsBra =('type_Sports Bra', max),
            fw_Type_Sweater =('type_Sweater', max),
            fw_Type_Other =('type_Other', max)).reset_index()

customer_cltv_first_week.head(2)

Now we merge them together to create the last table that we will need to work on our analysis. 

In [ ]:
## We merge both together and output our final table that contains information on all the purchases.
merged_cltv = customer_cltv.merge(customer_cltv_first_week, left_on = 'CustomerID', right_on = 'CustomerID').sort_values('CustomerID')
merged_cltv = merged_cltv[['CustomerID','CLTV_Total_Spent','CLTV_one_Week','Number_of_orders',
                           'Number_of_orders_one_week','First_Order_x','cltv_start_x','cltv_end_x',
                           'repeat_customer','Size_XSmall','Size_Small','Size_Medium','Size_Large',
                           'Size_XLarge','No_Size','Type_Barbell','Type_CableCuffs','Type_CropTops',
                           'Type_Hoodie','Type_Jacket','Type_Leggings','Type_LongSleeve','Type_ResistanceBands','Type_Shirts',
                           'Type_Shorts','Type_SportsBra','Type_Sweater','Type_Other','fw_Size_XSmall','fw_Size_Small',
                           'fw_Size_Medium','fw_Size_Large','fw_Size_XLarge','fw_No_Size','fw_Type_Barbell','fw_Type_CableCuffs',
                           'fw_Type_CropTops','fw_Type_Hoodie','fw_Type_Jacket','fw_Type_Leggings','fw_Type_LongSleeve','fw_Type_ResistanceBands',
                           'fw_Type_Shirts','fw_Type_Shorts','fw_Type_SportsBra','fw_Type_Sweater','fw_Type_Other']]
merged_cltv.head()

Finally, to get the last bit of information, we create two functions which will retrieve the item sku and frequency of purchase for each customer during the cltv and one week after first order periods. 

## Creating our FINAL CLTV TABLE by Customer

In [ ]:
merged_cltv.rename(columns={'CLTV_Total_Spent':'CLTV_one_year','CLTV_one_Week':'First_Week_Spent'}, inplace = True)
CLTV_avg = merged_cltv.CLTV_one_year.mean()
CLTV_avg_one_week = merged_cltv.First_Week_Spent.mean()
CLTV_avg_repeat = merged_cltv['CLTV_one_year'][merged_cltv['repeat_customer'] > 0].mean()
CLTV_avg_repeat_one_week = merged_cltv['First_Week_Spent'][merged_cltv['repeat_customer'] > 0].mean()
customer_count = merged_cltv['CustomerID'].count()
repeat_customer_count = merged_cltv['CustomerID'][merged_cltv['repeat_customer'] >0].count()

print('The number of customers with a one year CLTV is: ' + str(customer_count))
print('The number of repeat customers within the one year CLTV is: ' + str(repeat_customer_count))
print('The one year CLTV of customers is: $' + str(round(CLTV_avg,2)))
print('The first week CLTV of customers is: $' + str(round(CLTV_avg_one_week,2)))
print('The one year CLTV of repeat customers is: $' + str(round(CLTV_avg_repeat,2)))
print('The first week CLTV of repeat customers is: $' + str(round(CLTV_avg_repeat_one_week,2)))

## First Week CLTV Table Final

In [ ]:
merged_cltv_fw = merged_cltv
merged_cltv_fw.head(2)

In [ ]:
merged_cltv_fw['CLTV_post_firstweek'] = merged_cltv_fw['CLTV_one_year'] - merged_cltv_fw['First_Week_Spent']
merged_cltv_fw.drop(columns=['CLTV_one_year','repeat_customer','Number_of_orders','First_Order_x','cltv_start_x','cltv_end_x','Size_XSmall','Size_Small',
                             'Size_Medium','Size_Large','Size_XLarge','No_Size','Type_Barbell','Type_CableCuffs',
                             'Type_CropTops','Type_Hoodie','Type_Jacket','Type_Leggings','Type_LongSleeve','Type_Other',
                             'Type_ResistanceBands','Type_Shirts','Type_Shorts','Type_SportsBra','Type_Sweater'],inplace =True)
merged_cltv_fw.head(2)

In [ ]:
def order_range(row):
    if row['CLTV_post_firstweek'] <= 50 :
        return '<$50'
    elif row['CLTV_post_firstweek'] <= 100 :
        return '$50-100' 
    elif row['CLTV_post_firstweek'] <= 200: 
        return '$100-200'
    else:
        return '>$200'
merged_cltv_fw['CLTV_Range'] = merged_cltv_fw.apply(order_range, axis=1)
merged_cltv_fw.head(2)

# EDA 

In [ ]:
_ = plt.hist('CLTV_Range', data =merged_cltv_fw )

# Modeling

In [ ]:
sns.regplot(data = merged_cltv_fw, x ='First_Week_Spent',y='CLTV_post_firstweek')
plt.show()

In [ ]:
X = merged_cltv_fw[['First_Week_Spent']]
y = merged_cltv_fw[['CLTV_post_firstweek']]
 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.25, random_state =0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
rModel = linear_model.LinearRegression(normalize = True)

In [ ]:
rModel.fit(X_train, y_train)

In [ ]:
print(rModel.score(X_train,y_train))

In [ ]:
y_pred = rModel.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.plot([x for x in range(0,900)],[x for x in range(0,900)], color='r')
plt.show()

In [ ]:
merged_cltv_fw

In [ ]:
X = merged_cltv_fw.drop(['CustomerID','CLTV_Range','CLTV_post_firstweek'],axis=1)
X = sm.add_constant(X)
y = merged_cltv_fw['CLTV_post_firstweek']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.25, random_state = 123)

In [ ]:
rmodel = sm.OLS(y_train, X_train)
results =rmodel.fit()

In [ ]:
results.summary()

In [ ]:
y_pred = results.predict(X_test)
plt.scatter(y_test, y_pred)

# Add a line for perfect correlation
plt.plot([x for x in range(0,1200)],[x for x in range(0,1200)], color='g')

# Label it nicely
plt.show()

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())

y_pred = results.predict(X_test)

matches = pd.DataFrame(y_test)
matches.rename(columns = {'CLTV_post_firstweek':'actual'}, inplace=True)
matches["predicted"] = y_pred

rmse(matches["actual"], matches["predicted"])

In [ ]:
matches['difference'] = (matches['actual'] - matches['predicted'])**2
matches['difference_not_squared'] = abs((matches['actual'] - matches['predicted']))

In [ ]:
MAE = np.mean(matches.difference_not_squared)
print(MAE)

## Creating our FINAL CLTV TABLE by Item Ordered

In [ ]:
print('We start with ' +str(raw_data['Lineitem name'].count()) + ' rows in the raw data.')

In [ ]:
## After merging our tables, 
fw_items = raw_data[['OrderID','CustomerID','Transaction Date', 'Subtotal','Discount Code','Discount Amount','Lineitem quantity','Lineitem name','Lineitem price','Billing City','Billing Zip','Billing Province']]
fw_items[['Transaction Date','Billing City','Billing Zip','Billing Province']] = fw_items[['Transaction Date','Billing City','Billing Zip','Billing Province']].fillna(method='ffill')
fw_items['Billing Zip'] = fw_items['Billing Zip'].apply(lambda x: str.replace(x,"'",''))
fw_items = fw_items.merge(customer_cltv_first_week, left_on='CustomerID', right_on ='CustomerID')
print('After merging with the cltv_first_week, the total items are : ' + str(fw_items['Lineitem name'].count()))
print('Total unique orders : ' + str(fw_items.OrderID.nunique()))
print('Total unique customers: ' + str(fw_items.CustomerID.nunique()))

In [ ]:
fw_items  = fw_items[(fw_items['Transaction Date'] >= fw_items['First_Order']) & (fw_items['Transaction Date'] <= fw_items['cltv_start'])]
def order_range(row):
    if row['Subtotal'] <= 50 :
        return '<$50'
    elif row['Subtotal'] <= 100 :
        return '$50-100' 
    elif row['Subtotal'] <= 200: 
        return '$100-200'
    elif type(row['Subtotal']) == float and pd.isna(row['Subtotal']): 
        return 'Repeat order'
    else:
        return '>$200'
fw_items['Order_Range'] = fw_items.apply(order_range, axis=1)

print('Total items : ' + str(fw_items['Lineitem name'].count()))
print('Total unique orders : ' + str(fw_items.OrderID.nunique()))
print('Total customers : ' + str(fw_items.CustomerID.count()))
print('Total unique customers: ' + str(fw_items.CustomerID.nunique()))

fw_items.head(3)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax =sns.histplot(x='Order_Range', data = fw_items)
ax.set_title('Distribution of orders', fontsize =30)
ax.set_xlabel("Order_Range",fontsize=14)
ax.set_ylabel("Total",fontsize=14)
plt.show()

## CLTV Table by Order: 

In [ ]:
orders.head(2)

In [ ]:
orders_cltv_fw = orders.merge(customer_cltv_first_week, left_on='CustomerID', right_on ='CustomerID')
orders_cltv_fw  = orders_cltv_fw[(orders_cltv_fw['Transaction_Date'] >= orders_cltv_fw['First_Order']) & (orders_cltv_fw['Transaction_Date'] <= orders_cltv_fw['cltv_start'])]
print('Total Unique Orders in first week: ' +str(orders_cltv_fw.OrderID.count()))
orders_cltv_fw.head(2)

In [ ]:
def order_range(row):
    if row['Total_Spent'] <= 50 :
        return '<$50'
    elif row['Total_Spent'] <= 100 :
        return '$50-100' 
    elif row['Total_Spent'] <= 200: 
        return '$100-200'
    elif type(row['Total_Spent']) == float and pd.isna(row['Total_Spent']): 
        return 'Repeat order'
    else:
        return '>$200'


orders_cltv_fw ['Order_Range'] = orders_cltv_fw.apply(order_range, axis=1)

print('Total unique orders : ' + str(orders_cltv_fw.OrderID.nunique()))
print('Total customers : ' + str(orders_cltv_fw.CustomerID.count()))
print('Total unique customers: ' + str(orders_cltv_fw.CustomerID.nunique()))
orders_cltv_fw.head(2)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax =sns.histplot(x='Order_Range', data = orders_cltv_fw)
ax.set_title('Distribution of Order Range', fontsize =30)
ax.set_xlabel("Order_Range",fontsize=14)
ax.set_ylabel("Total",fontsize=14)
plt.show()

# Modelling

In [ ]:
merged_cltv.head(2)

In [ ]:
sns.regplot(data = merged_cltv, x ='CLTV_one_Week',y='CLTV_Total_Spent')
plt.show()

In [ ]:
## Regression
X = merged_cltv[['CLTV_one_Week']]
y = merged_cltv[['CLTV_Total_Spent']]
 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.25, random_state =0)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
rModel = linear_model.LinearRegression(normalize = True)

In [ ]:
rModel.fit(X_train, y_train)

In [ ]:
print(rModel.score(X_train,y_train))

In [ ]:
y_pred = rModel.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.plot([x for x in range(0,900)],[x for x in range(0,900)], color='r')
plt.show()

In [ ]:
X = merged_cltv.drop(['CLTV_Total_Spent',"CustomerID", "First_Order_x","cltv_start_x","cltv_end_x","Products_Purchased",'Products_Purchased_first_week'],axis=1)
X = sm.add_constant(X)
y = merged_cltv[['CLTV_Total_Spent']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.25, random_state = 123)

In [ ]:
rmodel3 = sm.OLS(y_train, X_train)
results3 =rmodel3.fit()

In [ ]:
results3.summary()

In [ ]:
y_pred = results3.predict(X_test)
plt.scatter(y_test, y_pred)

# Add a line for perfect correlation
plt.plot([x for x in range(0,1200)],[x for x in range(0,1200)], color='g')

# Label it nicely
plt.show()

In [ ]:
# Define a function to check the RMSE. Remember the def keyword needed to make functions? 
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())

y_pred = results3.predict(X_test)

matches = pd.DataFrame(y_test)
matches.rename(columns = {'CLTV_Total_Spent':'actual'}, inplace=True)
matches["predicted"] = y_pred

rmse(matches["actual"], matches["predicted"])

In [ ]:
matches['difference'] = (matches['actual'] - matches['predicted'])**2
matches['difference_not_squared'] = abs((matches['actual'] - matches['predicted']))

In [ ]:
MAE = np.mean(matches.difference_not_squared)
print(MAE)

In [ ]:
sns.set(rc = {'figure.figsize':(40,30)})
sns.heatmap(merged_cltv.corr(), annot =True)
#sns.heatmap(merged_cltv.corr())
plt.show()